## Purpose: Try different models-- Part7.
### deep learning.

In [1]:
# import dependencies.
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

Using TensorFlow backend.


#### STEP1: Read in dataset.  Remove data from 2016-2019.
- data from 2016-2018 will be used to bs test the model.
- data from 2019 will be used to predict the winners of the 2019 WS.

In [2]:
# read in the data.
team_data = pd.read_csv("../../Resources/clean_data_1969.csv")
del team_data["Unnamed: 0"]
team_data.head()

,team,year,A,DP,E,G2,GS2,INN,PB,PO,...,SHO,SO1,SV,SVO,TBF,W,WHIP,WP,WPCT,winners
0,St. Louis Cardinals,2019,1033,114,43,104,936,8313.0,3,2771,...,4,895,33,42,3896,56,1.29,21,0.538,0
1,Arizona Diamondbacks,2019,1010,83,45,105,945,8538.0,2,2846,...,7,925,24,37,4001,53,1.28,35,0.505,0
2,Kansas City Royals,2019,990,105,45,106,954,8421.0,6,2807,...,5,816,24,41,4125,39,1.46,34,0.368,0
3,Houston Astros,2019,875,54,50,106,954,8589.0,6,2863,...,7,1074,27,42,3929,67,1.14,31,0.632,0
4,Tampa Bay Rays,2019,975,92,53,107,963,8760.0,11,2920,...,6,1037,26,43,3985,59,1.16,40,0.551,0


In [3]:
# remove data from 2016 through 2019.
team_data_new = team_data.loc[team_data["year"] < 2016]
team_data_new.head()

,team,year,A,DP,E,G2,GS2,INN,PB,PO,...,SHO,SO1,SV,SVO,TBF,W,WHIP,WP,WPCT,winners
120,San Francisco Giants,2015,1639,136,72,162,1458,13143.0,6,4381,...,11,1309,43,72,6048,87,1.21,40,0.537,0
121,Washington Nationals,2015,1425,142,73,162,1458,13137.0,17,4379,...,12,1476,46,60,6036,95,1.19,47,0.586,0
122,Houston Astros,2015,1599,135,77,162,1458,13212.0,18,4404,...,8,1396,44,64,6180,84,1.29,98,0.519,0
123,Detroit Tigers,2015,1537,148,75,161,1449,12852.0,5,4284,...,8,1232,47,66,6048,86,1.32,44,0.534,0
124,Boston Red Sox,2015,1427,139,75,162,1458,12957.0,37,4319,...,5,1362,43,61,6073,93,1.27,52,0.574,0


In [4]:
target = team_data_new["winners"]
features = team_data_new.drop({"team", "year", "winners"}, axis=1)
feature_columns = list(features.columns)
print (target.shape)
print (features.shape)
print (feature_columns)

(1266,)
(1266, 59)
['A', 'DP', 'E', 'G2', 'GS2', 'INN', 'PB', 'PO', 'TC', '2B', '3B', 'AB', 'AO', 'BB', 'CS', 'G', 'GDP', 'H', 'HBP', 'HR', 'IBB', 'NP_x', 'OBP', 'OPS_x', 'PA', 'R', 'RBI', 'SAC', 'SB', 'SF', 'SLG', 'SO', 'TB', 'XBH', 'BB1', 'BK', 'CG', 'ER', 'ERA', 'G1', 'GF', 'GS', 'H1', 'HB', 'HR1', 'IBB1', 'IP', 'L', 'OBP1', 'R1', 'SHO', 'SO1', 'SV', 'SVO', 'TBF', 'W', 'WHIP', 'WP', 'WPCT']


#### STEP2: Split and scale data.

In [5]:
# split into train and test sets.
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42, stratify=target)

# scale the data.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

# label-encode dataset.
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# convrt labels.
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


#### STEP3: Generate model.

In [7]:
# create model and add layers.
model = Sequential()
model.add(Dense(units=100, activation="relu", input_dim=59))
model.add(Dense(units=100, activation="relu"))
model.add(Dense(units=2, activation="softmax"))

# compile and fit the model.
model.compile(optimizer="adam",
             loss="categorical_crossentropy",
             metrics=["accuracy"])

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2)

Epoch 1/60
 - 1s - loss: 0.2134 - acc: 0.9600
Epoch 2/60
 - 0s - loss: 0.1435 - acc: 0.9642
Epoch 3/60
 - 0s - loss: 0.1307 - acc: 0.9642
Epoch 4/60
 - 0s - loss: 0.1218 - acc: 0.9642
Epoch 5/60
 - 0s - loss: 0.1123 - acc: 0.9642
Epoch 6/60
 - 0s - loss: 0.1033 - acc: 0.9642
Epoch 7/60
 - 0s - loss: 0.0950 - acc: 0.9663
Epoch 8/60
 - 0s - loss: 0.0874 - acc: 0.9663
Epoch 9/60
 - 0s - loss: 0.0872 - acc: 0.9694
Epoch 10/60
 - 0s - loss: 0.0695 - acc: 0.9705
Epoch 11/60
 - 0s - loss: 0.0677 - acc: 0.9726
Epoch 12/60
 - 0s - loss: 0.0566 - acc: 0.9779
Epoch 13/60
 - 0s - loss: 0.0454 - acc: 0.9800
Epoch 14/60
 - 0s - loss: 0.0414 - acc: 0.9821
Epoch 15/60
 - 0s - loss: 0.0319 - acc: 0.9905
Epoch 16/60
 - 0s - loss: 0.0280 - acc: 0.9937
Epoch 17/60
 - 0s - loss: 0.0209 - acc: 0.9979
Epoch 18/60
 - 0s - loss: 0.0178 - acc: 0.9968
Epoch 19/60
 - 0s - loss: 0.0148 - acc: 1.0000
Epoch 20/60
 - 0s - loss: 0.0134 - acc: 0.9989
Epoch 21/60
 - 0s - loss: 0.0119 - acc: 0.9979
Epoch 22/60
 - 0s - lo

In [8]:
# quantify the trained model.
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print (f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.5512785124665931, Accuracy: 0.9400630918587044


In [9]:
encoded_predictions = model.predict_classes(X_test_scaled[:60])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [10]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:60])}")

Predicted classes: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual Labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [ ]:
nope.